In [2]:
import pandas as pd
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [4]:
%run helper_functions

In [7]:
data_path = "/Users/eisukeokuda/DIW/folketinget/data/"

In [11]:
dspeech_tok1 = pd.read_parquet(data_path+'data_speech1_tok.parquet')
dspeech_tok2 = pd.read_parquet(data_path+'data_speech2_tok.parquet')
dspeech_tok = pd.concat([dspeech_tok1, dspeech_tok2]).reset_index(drop=True)

In [10]:
data_speech1 = pd.read_parquet(data_path + 'data_speech1.parquet')
data_speech2 = pd.read_parquet(data_path + 'data_speech2.parquet')
data_speech3 = pd.read_parquet(data_path + 'data_speech3.parquet')
#parMem = pd.read_parquet(data_path + 'parliament_members.parquet')

#dagenda = pd.concat([data_agenda1, data_agenda2, data_agenda3], axis=0)
dspeech = pd.concat([data_speech1, data_speech2, data_speech3], axis=0)

In [23]:
dspeech_tok = pd.merge(dspeech_tok, dspeech[["meeting_id", "agenda_item_id", "speech_item_id", "label"]], on=["meeting_id", "agenda_item_id", "speech_item_id"], how="right")

### Calculate Odds with Labeled Data

In [25]:
dspeech_tok["speech_item_tokenized"] = dspeech_tok["speech_item_tokenized"].apply(lambda x: eval(x))

In [26]:
df_C = dspeech_tok[dspeech_tok['label']=='C']
df_NC = dspeech_tok[dspeech_tok['label']=='NC']

unique_words_C_list = [word for tokens in df_C.speech_item_tokenized for word in tokens]
unique_words_NC_list = [word for tokens in df_NC.speech_item_tokenized for word in tokens]

unique_words_C_dict = Counter(unique_words_C_list)
unique_words_NC_dict = Counter(unique_words_NC_list)

unique_words_C_df = pd.DataFrame.from_dict(unique_words_C_dict, orient='index', columns=['word_count_C']).reset_index().rename(columns={'index': 'word'})
unique_words_NC_df = pd.DataFrame.from_dict(unique_words_NC_dict, orient='index', columns=['word_count_NC']).reset_index().rename(columns={'index': 'word'})


unique_words = pd.merge(unique_words_C_df, unique_words_NC_df, on='word', how='outer')
unique_words[['word_count_C', 'word_count_NC']] = unique_words[['word_count_C', 'word_count_NC']].fillna(0)
unique_words['frequency_C'] = unique_words['word_count_C'] / sum(unique_words_C_df['word_count_C'])
unique_words['frequency_NC'] = unique_words['word_count_NC'] / sum(unique_words_NC_df['word_count_NC'])
freq_1_NC = 1/unique_words['word_count_NC'].sum()
freq_1_C = 1/unique_words['word_count_C'].sum()

unique_words["odds_C"] = unique_words.apply(lambda row: calculate_odds(row['frequency_C'], row['frequency_NC'], 'C', freq_1_NC), axis=1)
unique_words["odds_NC"] = unique_words.apply(lambda row: calculate_odds(row['frequency_NC'], row['frequency_C'], 'NC', freq_1_C), axis=1)

### Save All Words with Odds_C > 2

In [38]:
unique_words.loc[unique_words["odds_C"] >= 2, ["word", "word_count_C", "odds_C"]].to_parquet(data_path+'final_C_words.parquet')